## Imports and datafraem


In [1]:
# DBscan from scratch
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons

# Load the data
old_df = pd.read_csv('new_dataset.csv')

# Print the first 5 rows of the data    
old_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1,1.0,0.715820,Sales,0.000000,0.250000,Life Sciences,0.333333,0.0,...,0.00,0.00,0.0,0.000,0.0,0.00,0.15,0.222222,0.000000,0.344828
1,0.814978,0,0.5,0.185735,Research & Development,0.275862,0.157895,Life Sciences,0.727273,1.0,...,0.92,1.00,0.5,0.025,0.5,0.75,0.25,0.388889,0.066667,0.482759
2,0.613136,1,1.0,0.915934,Research & Development,0.068966,0.380435,Other,1.000000,1.0,...,0.60,0.50,0.0,0.025,0.5,0.75,0.00,0.000000,0.000000,0.000000
3,0.545855,0,0.5,0.928610,Research & Development,0.103448,0.793478,Life Sciences,1.000000,0.0,...,0.44,0.75,0.0,0.025,0.5,0.75,0.20,0.388889,0.200000,0.000000
4,0.444950,0,1.0,0.394188,Research & Development,0.068966,0.173913,Medical,0.181818,1.0,...,0.48,1.00,0.5,0.025,0.5,0.75,0.05,0.111111,0.133333,0.137931


In [2]:
df = old_df.copy()
columns = ['Department', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']
#target based encoding, each category of the column will bear the mean
for column in columns:
        if column == 'Department':
                df[column] = df.groupby(column)['JobSatisfaction'].transform('mean')
        elif column == 'EducationField':
                df[column] = df.groupby(column)['HourlyRate'].transform('mean')
        elif column == 'JobRole':
                df[column] = df.groupby(column)['HourlyRate'].transform('mean')
        elif column == 'MaritalStatus':
                df[column] = df.groupby(column)['WorkLifeBalance'].transform('mean')
        elif column == 'OverTime':
                df[column] = df.groupby(column)['YearsSinceLastPromotion'].transform('mean')

df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1,1.0,0.715820,0.673292,0.000000,0.250000,0.667702,0.333333,0.0,...,0.00,0.00,0.0,0.000,0.0,0.00,0.15,0.222222,0.000000,0.344828
1,0.814978,0,0.5,0.185735,0.666719,0.275862,0.157895,0.667702,0.727273,1.0,...,0.92,1.00,0.5,0.025,0.5,0.75,0.25,0.388889,0.066667,0.482759
2,0.613136,1,1.0,0.915934,0.666719,0.068966,0.380435,0.623913,1.000000,1.0,...,0.60,0.50,0.0,0.025,0.5,0.75,0.00,0.000000,0.000000,0.000000
3,0.545855,0,0.5,0.928610,0.666719,0.103448,0.793478,0.667702,1.000000,0.0,...,0.44,0.75,0.0,0.025,0.5,0.75,0.20,0.388889,0.200000,0.000000
4,0.444950,0,1.0,0.394188,0.666719,0.068966,0.173913,0.653232,0.181818,1.0,...,0.48,1.00,0.5,0.025,0.5,0.75,0.05,0.111111,0.133333,0.137931


In [3]:
class DBSCAN: 
    def __init__(self, eps, min_pts, data):
        self.eps = eps
        self.min_pts = min_pts
        self.data = data
        self.clusters = []
        self.noise = []
        self.core_pts = []
        self.visited = []
        self.clustered = []
        self.cluster_num = 0
        self.clustered_pts = []
        
    def _distance(self, p1, p2):
        result = 0
        for i in range(len(p1)):
            if(type(p1[i]) == str or type(p2[i]) == str):
                if(p1[i] != p2[i]):
                    result += 1
            else : result += (p1[i] - p2[i]) ** 2
        return math.sqrt(result)
        #return math.sqrt(sum([(a - b) ** 2 for a, b in zip(p1, p2)]))
    
    def _region_query(self, point):
        neighbors = []
        for i in range(len(self.data)):
            if self._distance(point, self.data[i]) < self.eps:
                neighbors.append(i)
        return neighbors
    
    def _expand_cluster(self, point, neighbors):
        self.clusters[self.cluster_num].append(point)
        self.clustered.append(point)
        self.visited.append(point)
        for i in neighbors:
            if i not in self.visited:
                self.visited.append(i)
                new_neighbors = self._region_query(self.data[i])
                if len(new_neighbors) >= self.min_pts:
                    neighbors += new_neighbors
            if i not in self.clustered:
                self.clusters[self.cluster_num].append(i)
                self.clustered.append(i)
                
    def fit(self):
        for i in range(len(self.data)):
            if i not in self.visited:
                self.visited.append(i)
                neighbors = self._region_query(self.data[i])
                if len(neighbors) < self.min_pts:
                    self.noise.append(i)
                else:
                    self.clusters.append([])
                    self._expand_cluster(i, neighbors)
                    self.cluster_num += 1
                    
    def get_clusters(self):
        return self.clusters
    
    def get_noise(self):
        return self.noise



In [4]:
Y = df['Attrition'].values
df = df.drop(['Attrition'], axis=1)
X = df.values

In [5]:
dbscan = DBSCAN(eps=2.135, min_pts=1, data=X)
dbscan.fit()
clusters = dbscan.get_clusters()
noise = dbscan.get_noise()

In [6]:
len(dbscan.get_clusters())

1

In [7]:
dbscan.get_clusters()

[[0,
  2,
  3,
  4,
  5,
  6,
  11,
  13,
  14,
  18,
  19,
  22,
  24,
  25,
  29,
  30,
  32,
  34,
  35,
  36,
  37,
  38,
  40,
  42,
  44,
  47,
  48,
  50,
  51,
  52,
  54,
  55,
  57,
  62,
  64,
  65,
  66,
  68,
  69,
  70,
  72,
  74,
  75,
  78,
  79,
  81,
  83,
  84,
  85,
  92,
  94,
  95,
  96,
  102,
  103,
  106,
  107,
  109,
  113,
  114,
  115,
  117,
  118,
  122,
  124,
  125,
  128,
  129,
  130,
  131,
  132,
  133,
  137,
  139,
  140,
  141,
  142,
  143,
  145,
  146,
  147,
  149,
  150,
  152,
  153,
  154,
  156,
  157,
  158,
  159,
  162,
  163,
  165,
  166,
  167,
  168,
  169,
  172,
  174,
  175,
  176,
  177,
  179,
  180,
  182,
  183,
  186,
  188,
  191,
  192,
  195,
  196,
  198,
  203,
  204,
  206,
  207,
  212,
  214,
  215,
  217,
  219,
  221,
  222,
  228,
  229,
  230,
  232,
  233,
  234,
  235,
  236,
  238,
  239,
  240,
  245,
  246,
  248,
  252,
  253,
  255,
  258,
  260,
  263,
  264,
  274,
  275,
  276,
  278,
  279,
  282,
  

In [8]:
len(dbscan.get_noise())

0

In [9]:
dbscan.get_noise()

[]

In [10]:
# # sklearn dbscan
# from sklearn.cluster import DBSCAN
# from sklearn.preprocessing import StandardScaler

# X = df.values
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# dbscan = DBSCAN(eps=0.5, min_samples=5)
# clusters = dbscan.fit_predict(X_scaled)

# clusters

In [25]:
class AGNES:
    def __init__(self, n_clusters=2, linkage='average'):
        self.n_clusters = n_clusters
        self.linkage = linkage
        self.labels_ = None
        self.cluster_centers_ = None
        self.n_leaves = None
        self.results = {}

    def fit(self, X):
        #X = np.array([self.string_to_numerical(x) for x in X])
        
        self.n_leaves = X.shape[0]
        self.labels_ = np.arange(self.n_leaves)
        self.cluster_centers_ = X.copy()
        self.results[self.n_leaves+1] = [self.labels_.copy(), self.cluster_centers_.copy()]

        # while self.n_leaves >= self.n_clusters:
        #     self.merge()

        while self.n_leaves >= 1:
            self.merge()
            self.results[self.n_leaves+1] = [self.labels_.copy(), self.cluster_centers_.copy()]
        return self

    def merge(self):
        dist = self.distance(self.cluster_centers_)
        i, j = np.unravel_index(dist.argmin(), dist.shape)
        self.cluster_centers_[i] = self._linkage(i, j)
        self.cluster_centers_ = np.delete(self.cluster_centers_, j, axis=0)
        self.labels_[self.labels_ == j] = i
        self.labels_[self.labels_ > j] -= 1
        self.n_leaves -= 1
    
    def string_to_numerical(self,arr):
        """Convert a string to a numerical value"""
        # Convert the string to a list of ASCII values
        new_arr = []
        for s in arr:
            #print(type(s))
            if (type(s) == str):
                ascii_values = [ord(c) for c in s]
                # Convert the list of ASCII values to a numpy array
                ascii_array = np.array(ascii_values)
                # Return the sum of the array
                new_arr.append(ascii_array.sum())
            else: new_arr.append(s)
        return np.array(new_arr)

    def distance(self, X):
        return np.sqrt(-2 * np.dot(X, X.T) + np.sum(X ** 2, axis=1) + np.sum(X ** 2, axis=1)[:, np.newaxis])

    def _distance(self, p1, p2):
        result = 0
        for i in range(len(p1)):
            if(type(p1[i]) == str or type(p2[i]) == str):
                if(p1[i] != p2[i]):
                    result += 1
            else : result += (p1[i] - p2[i]) ** 2
        return math.sqrt(result)

    def distance2(self, X):
        distances = []
        for i in range(len(X)):
            for j in range(len(X)):
                distances.append(self._distance(X[i], X[j]))
        return np.array(distances).reshape(len(X), len(X))
                
    

    def _linkage(self, i, j):
        if self.linkage == 'average':
            # linkage = []
            # for i in range (len(self.cluster_centers_[i])):
            #     if type(self.cluster_centers_[i][i]) == str:
            #         if self.cluster_centers_[i][i] != self.cluster_centers_[j][i]:
            #             linkage.append(0)
            #         else: linkage.append(1)
            #     else: linkage.append((self.cluster_centers_[i][i] + self.cluster_centers_[j][i]))
            # linkage = np.array(linkage)
            # return linkage / 2
            return (self.cluster_centers_[i] + self.cluster_centers_[j]) / 2
        elif self.linkage == 'single':
            return np.minimum(self.cluster_centers_[i], self.cluster_centers_[j])
        elif self.linkage == 'complete':
            return np.maximum(self.cluster_centers_[i], self.cluster_centers_[j])
        else:
            raise ValueError('Unknown linkage method: {}'.format(self.linkage))

    def predict(self, X):
        return self.labels_

    def fit_predict(self, X):
        self.fit(X)
        return self.predict(X)

    def get_results(self):
        return self.results

In [38]:
agnes = AGNES(n_clusters=2, linkage='single')
res = agnes.fit_predict(X)
res

C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in sqrt


array([0, 0, 0, ..., 0, 0, 0])

In [39]:
results = agnes.get_results()

In [40]:
for each in results[2][0]:
        print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [41]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

def precision(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    return tp / (tp + fp)

def sensitivity(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    return tp / (tp + fn)

def specificity(y_true, y_pred):
    tn = np.sum((1 - y_true) * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    return tn / (tn + fp)

def f_score(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * precision * recall / (precision + recall)

# confusion matrix containing true positives, false negatives, false positives and true negatives
def confusion_matrix(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    tn = np.sum((1 - y_true) * (1 - y_pred))
    return np.array([[tp, fn], [fp, tn]])

In [42]:
print(f"accuracy = {accuracy(Y, results[2][0])}")
print(f"precision = {precision(Y, results[2][0])}")
print(f"sensitivity = {sensitivity(Y, results[2][0])}")
print(f"specificity = {specificity(Y, results[2][0])}")
print(f"f_score = {f_score(Y, results[2][0])}")
print(f"confusion_matrix = \n{confusion_matrix(Y, results[2][0])}")

accuracy = 0.3054421768707483
precision = 0.16265060240963855
sensitivity = 0.7974683544303798
specificity = 0.21086780210867803
f_score = 0.270192994996426
confusion_matrix = 
[[189  48]
 [973 260]]


In [43]:
dbscan_res = {}
val = 0
for cluster in dbscan.get_clusters():
        for i in cluster:
                dbscan_res[i] = val
        val += 1

In [44]:
# order dict by key
dbscan_res = {k: dbscan_res[k] for k in sorted(dbscan_res)}
dbscan_res = list(dbscan_res.values())

In [45]:
dbscan_res = np.array(dbscan_res)

In [46]:
print(f"accuracy = {accuracy(Y, dbscan_res)}")
print(f"precision = {precision(Y, dbscan_res)}")
print(f"sensitivity = {sensitivity(Y, dbscan_res)}")
print(f"specificity = {specificity(Y, dbscan_res)}")
print(f"f_score = {f_score(Y, dbscan_res)}")
print(f"confusion_matrix = \n{confusion_matrix(Y, dbscan_res)}")


accuracy = 0.8387755102040816
precision = nan
sensitivity = 0.0
specificity = 1.0
f_score = nan
confusion_matrix = 
[[   0  237]
 [   0 1233]]


C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  
C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in longlong_scalars


In [47]:
# sklearn agglomerative clustering 
from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average')
agg_res = agg.fit_predict(X)

In [48]:
for each in agg_res:
        print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [49]:
print(f"accuracy = {accuracy(Y, agg_res)}")
print(f"precision = {precision(Y, agg_res)}")
print(f"sensitivity = {sensitivity(Y, agg_res)}")
print(f"specificity = {specificity(Y, agg_res)}")
print(f"f_score = {f_score(Y, agg_res)}")
print(f"confusion_matrix = \n{confusion_matrix(Y, agg_res)}")

accuracy = 0.8367346938775511
precision = 0.0
sensitivity = 0.0
specificity = 0.9975669099756691
f_score = nan
confusion_matrix = 
[[   0  237]
 [   3 1230]]


C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
